# Batch normalization

### Issues with training Deep Neural Networks 

- There are 2 major issues 1) Internal Covariate shift, 2) Vanishing Gradient

### Internal Covariate shift

- The concept of covariate shift pertains to the change that occurs in the distribution of the input to a learning system. In deep networks, this distribution can be influenced by parameters across all input layers. Consequently, even minor changes in the network can have a significant impact on its output. This effect gets magnified as the signal propagates through the network, which can result in a shift in the distribution of the inputs to internal layers. This phenomenon is known as internal covariate shift.

- When inputs are whitened (i.e., have zero mean and unit variance) and are uncorrelated, they tend to converge faster during training. However, internal covariate shift can have the opposite effect, as it introduces changes to the distribution of inputs that can slow down convergence. Therefore, to mitigate this effect, techniques like batch normalization have been developed to normalize the inputs to each layer in the network based on statistics of the current mini-batch.

### Vanishing Gradient

- Saturating non-linearities such as sigmoid or tanh are not suitable for deep networks, as the signal tends to get trapped in the saturation region as the network grows deeper. This makes it difficult for the network to learn and can result in slow convergence during training. To overcome this problem we can use the following.

- Non-linearities like ReLU which do not saturate.
- Smaller learning rates
- Careful initializations
---
### What is Normalization?

- Normalization in deep learning refers to the process of transforming the input or output of a layer in a neural network to improve its performance during training. The most common type of normalization used in deep learning is batch normalization, which normalizes the activations of a layer for each mini-batch during training.
---
### What is batch normalization?

- Batch normalization is a technique in deep learning that helps to standardize and normalize the input to each layer of a neural network by adjusting and scaling the activations. The idea behind batch normalization is to normalize the inputs to a layer to have zero mean and unit variance across each mini-batch of the training data.

### Steps involved in batch normalization

1) During training, for each mini-batch of data, compute the mean and variance of the activations of each layer. This can be done using the following formulas:

- Mean: $\mu_B = \frac{1}{m} \sum_{i=1}^m x_i$

- Variance: $\sigma_B^2 = \frac{1}{m} \sum_{i=1}^m (x_i - \mu_B)^2$

- Here, $m$ is the size of the mini-batch, and $x_i$ is the activation of the $i$-th neuron in the layer.

2) Normalize the activations of each layer in the mini-batch using the following formula:

- $\hat{x_i} = \frac{x_i - \mu_B}{\sqrt{\sigma_B^2 + \epsilon}}$
Here, $\epsilon$ is a small constant added for numerical stability.

3) Scale and shift the normalized activations using the learned parameters $\gamma$ and $\beta$, respectively:

- $y_i = \gamma \hat{x_i} + \beta$
- The parameters $\gamma$ and $\beta$ are learned during training using backpropagation.

4) During inference, the running mean and variance of each layer are used for normalization instead of the mini-batch statistics. These running statistics are updated using a moving average of the mini-batch statistics during training.
---
### The benefits of batch normalization include:

- Improved training performance: Batch normalization reduces the internal covariate shift, which is the change in the distribution of the activations of each layer due to changes in the distribution of the inputs. This allows the network to converge faster and with more stable gradients.

- Regularization: Batch normalization acts as a form of regularization by adding noise to the activations of each layer, which can help prevent overfitting.

- Increased robustness: Batch normalization makes the network more robust to changes in the input distribution, which can help improve its generalization performance.
---
### Code example for batch normalization

```python
import tensorflow as tf

# Define a fully connected layer
fc_layer = tf.keras.layers.Dense(units=128, activation='relu')

# Add batch normalization to the layer
bn_layer = tf.keras.layers.BatchNormalization()

# Define the model with the layer and batch normalization
model = tf.keras.models.Sequential([fc_layer, bn_layer])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy')

# Train the model
model.fit(x_train, y_train, batch_size=32, epochs=10)
```

- In the above code,the tf.keras.layers.BatchNormalization() layer is added after the fully connected layer to normalize the output before passing it to the activation function. The model.fit() function is then used to train the model using batch normalization.

In [54]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 
import time 
plt.style.use("fivethirtyeight")
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [55]:
(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [56]:
tf.random.set_seed(42) # for reproducibility
np.random.seed(42) # for reproducibility

LAYERS = [
          tf.keras.layers.Flatten(input_shape=[28, 28]),
          tf.keras.layers.Dense(300, kernel_initializer="he_normal", name="hiddenLayer1"),
          tf.keras.layers.LeakyReLU(),
          tf.keras.layers.Dense(100, kernel_initializer="he_normal" ,name="hiddenLayer2"),
          tf.keras.layers.LeakyReLU(),
          tf.keras.layers.Dense(10, activation="softmax", name="outputLayer")
        ]

model = tf.keras.models.Sequential(LAYERS)


In [57]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [58]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_8 (Flatten)         (None, 784)               0         
                                                                 
 hiddenLayer1 (Dense)        (None, 300)               235500    
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 300)               0         
                                                                 
 hiddenLayer2 (Dense)        (None, 100)               30100     
                                                                 
 leaky_re_lu_7 (LeakyReLU)   (None, 100)               0         
                                                                 
 outputLayer (Dense)         (None, 10)                1010      
                                                                 
Total params: 266610 (1.02 MB)
Trainable params: 26661

In [59]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 9s 4ms/step - loss: 1.2590 - accuracy: 0.6087 - val_loss: 0.8517 - val_accuracy: 0.7276
Epoch 2/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.7709 - accuracy: 0.7473 - val_loss: 0.6905 - val_accuracy: 0.7774
Epoch 3/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.6634 - accuracy: 0.7821 - val_loss: 0.6285 - val_accuracy: 0.7914
Epoch 4/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.6078 - accuracy: 0.8003 - val_loss: 0.5780 - val_accuracy: 0.8120
Epoch 5/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.5727 - accuracy: 0.8105 - val_loss: 0.5499 - val_accuracy: 0.8206
Epoch 6/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.5476 - accuracy: 0.8168 - val_loss: 0.5296 - val_accuracy: 0.8266
Epoch 7/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.5288 - accuracy: 0.8208 - val_loss: 0.5125 - val_accuracy:

In [60]:
y_pred =  model.predict(X_test[:3])

1/1 [==============================] - 0s 82ms/step


In [61]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.5138 - accuracy: 0.8228


[0.5138227939605713, 0.8227999806404114]

### Conclusion
- Runtime = 1m 4s
- Accuracy = 0.8321

## After applying  Batch Normalization

In [62]:
del model 

In [63]:
LAYERS = [
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(300, activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10, activation="softmax")
]

model = tf.keras.models.Sequential(LAYERS)

In [64]:
## Normalization elements are added to the model 2 * 2 because one for gamma and one for beta, mue and sigma
784*2*2, 300*2*2, 100*2*2, (784*2*2 + 300*2*2 + 100*2*2)/2

(3136, 1200, 400, 2368.0)

In [65]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_9 (Flatten)         (None, 784)               0         
                                                                 
 batch_normalization_6 (Bat  (None, 784)               3136      
 chNormalization)                                                
                                                                 
 dense_6 (Dense)             (None, 300)               235500    
                                                                 
 batch_normalization_7 (Bat  (None, 300)               1200      
 chNormalization)                                                
                                                                 
 dense_7 (Dense)             (None, 100)               30100     
                                                                 
 batch_normalization_8 (Bat  (None, 100)              

In [66]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [67]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 10s 5ms/step - loss: 0.8503 - accuracy: 0.7143 - val_loss: 0.5531 - val_accuracy: 0.8162
Epoch 2/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.5709 - accuracy: 0.8038 - val_loss: 0.4767 - val_accuracy: 0.8392
Epoch 3/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5165 - accuracy: 0.8207 - val_loss: 0.4423 - val_accuracy: 0.8528
Epoch 4/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4793 - accuracy: 0.8314 - val_loss: 0.4220 - val_accuracy: 0.8540
Epoch 5/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4556 - accuracy: 0.8404 - val_loss: 0.4069 - val_accuracy: 0.8604
Epoch 6/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.4385 - accuracy: 0.8451 - val_loss: 0.3954 - val_accuracy: 0.8640
Epoch 7/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.4234 - accuracy: 0.8500 - val_loss: 0.3853 - val_accuracy

In [68]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.4071 - accuracy: 0.8542


[0.4071011543273926, 0.854200005531311]

### Conclusion
- Runtime = 1m 9 sec
- Accuracy = 0.8622